In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score
from logitplots import plt_confusion_matrix, plt_decision_boundaries, plt_correlation_matrix

In [2]:
df= pd.read_csv("chipVariantCalling_run2.tsv", sep='\t')
df = df.drop_duplicates()
df = df.dropna()
df = df.drop(df[df.chipOrControl == 'Unknown'].index)

C:\Users\patty\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (10,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Create Dummy's for Categorical Variables

In [3]:
categories = df['SYMBOL'].unique()
lencat = len(categories) 
for a in categories: 

    df[a] = pd.Series(df['SYMBOL']==a).astype(int)

categories = df['sampleTimePt'].unique()
lencat = len(categories) 
for a in categories: 

    df[a] = pd.Series(df['sampleTimePt']==a).astype(int)
    
categories = df['gender'].unique()
lencat = len(categories) 
for a in categories: 

    df[a] = pd.Series(df['gender']==a).astype(int)
    
categories = df['IMPACT'].unique()
lencat = len(categories) 
for a in categories: 

    df[a] = pd.Series(df['IMPACT']==a).astype(int)

df = df.drop(['SYMBOL', 'sampleTimePt','gender','IMPACT'], axis = 1)

## Remove Identifiers from Data Frame we Will use to Make Models

In [4]:
final_df = df.drop(['d.barcode','MSID'], axis = 1)
final_df = final_df.reset_index(drop = True)
final_df.head(6)

,DP,VD,AF,HIAF,loci,BIAS,REFBIAS,VARBIAS,QUAL,ODDRATIO,...,SRSF2,RP11-661A12.9,TLK2,SLC5A10,Baseline,Y3,Male,Female,HIGH,MODERATE
0,2036,6,0.0029,0.0044,chr1:1747193_A/G,2:2,940:982,3:3,35.0,1.04467,...,0,0,0,0,1,0,1,0,1,0
1,2036,7,0.0034,0.0035,chr1:1747198_G/A,2:2,1007:1019,3:4,35.3,1.31745,...,0,0,0,0,1,0,1,0,0,1
2,2040,5,0.0025,0.0021,chr1:1747220_C/T,2:2,1002:1014,2:3,31.8,1.48196,...,0,0,0,0,1,0,1,0,0,1
3,2041,6,0.0029,0.0025,chr1:1747238_G/A,2:2,1008:1025,3:3,32.7,1.01686,...,0,0,0,0,1,0,1,0,0,1
4,2037,5,0.0025,0.0028,chr1:1747256_T/C,2:2,920:1028,2:3,32.2,1.34221,...,0,0,0,0,1,0,1,0,0,1
5,2037,5,0.0025,0.0018,chr1:1747259_T/C,2:2,862:1029,2:3,24.2,1.25641,...,0,0,0,0,1,0,1,0,0,1


## Convert ratios to Decimals

In [5]:
the_list = []                             # create an empty list
for i in range(len(final_df['BIAS'])):      # loop over all values
    ratio = final_df['BIAS'][i]     # read the ratio
    first = ''
    second = ''
    index = ratio.index(':')              # find the index of ':' in 'ratio'
    for a in range(0, index):             # get the value of the first part of the ratio
        first += ratio[a]
    for b in range(index+1, len(ratio)):  # get the value of the second part of the ratio
        second += ratio[b]
    total = int(first) + int(second)      # total = first + second
    if total == 0:                        # if the total = 0, then set decimal to be 10.0
        the_list += [10.0]
    else:
        decimal = int(first) / total      # else, set decimal to be first/total
        the_list += [decimal]


final_df['bias_decimal'] = the_list

In [6]:
the_list = []                             # create an empty list
for i in range(len(final_df['REFBIAS'])):      # loop over all values
    ratio = final_df['REFBIAS'][i]     # read the ratio
    first = ''
    second = ''
    index = ratio.index(':')              # find the index of ':' in 'ratio'
    for a in range(0, index):             # get the value of the first part of the ratio
        first += ratio[a]
    for b in range(index+1, len(ratio)):  # get the value of the second part of the ratio
        second += ratio[b]
    total = int(first) + int(second)      # total = first + second
    if total == 0:                        # if the total = 0, then set decimal to be 10.0
        the_list += [10.0]
    else:
        decimal = int(first) / total      # else, set decimal to be first/total
        the_list += [decimal]


final_df['REFBIAS'] = the_list

In [7]:
the_list = []                             # create an empty list
for i in range(len(final_df['VARBIAS'])):      # loop over all values
    ratio = final_df['VARBIAS'][i]     # read the ratio
    first = ''
    second = ''
    index = ratio.index(':')              # find the index of ':' in 'ratio'
    for a in range(0, index):             # get the value of the first part of the ratio
        first += ratio[a]
    for b in range(index+1, len(ratio)):  # get the value of the second part of the ratio
        second += ratio[b]
    total = int(first) + int(second)      # total = first + second
    if total == 0:                        # if the total = 0, then set decimal to be 10.0
        the_list += [10.0]
    else:
        decimal = int(first) / total      # else, set decimal to be first/total
        the_list += [decimal]


final_df['VARBIAS'] = the_list

## Normalisation

In [8]:
mean = final_df['DP'].mean()
SD = final_df['DP'].mean()
the_list = []  
for i in final_df['DP']:
    (i - mean)/SD
    

## Random Forest

In [9]:
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression 

In [10]:
final_df.head()

,DP,VD,AF,HIAF,loci,BIAS,REFBIAS,VARBIAS,QUAL,ODDRATIO,...,RP11-661A12.9,TLK2,SLC5A10,Baseline,Y3,Male,Female,HIGH,MODERATE,bias_decimal
0,2036,6,0.0029,0.0044,chr1:1747193_A/G,2:2,0.489074,0.500000,35.0,1.04467,...,0,0,0,1,0,1,0,1,0,0.5
1,2036,7,0.0034,0.0035,chr1:1747198_G/A,2:2,0.497038,0.428571,35.3,1.31745,...,0,0,0,1,0,1,0,0,1,0.5
2,2040,5,0.0025,0.0021,chr1:1747220_C/T,2:2,0.497024,0.400000,31.8,1.48196,...,0,0,0,1,0,1,0,0,1,0.5
3,2041,6,0.0029,0.0025,chr1:1747238_G/A,2:2,0.495819,0.500000,32.7,1.01686,...,0,0,0,1,0,1,0,0,1,0.5
4,2037,5,0.0025,0.0028,chr1:1747256_T/C,2:2,0.472279,0.400000,32.2,1.34221,...,0,0,0,1,0,1,0,0,1,0.5


In [11]:
X = final_df.drop(['chipOrControl', 'loci','BIAS'], axis = 1) 
y = final_df['chipOrControl']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.8, random_state=0)
rf = RandomForestClassifier(n_estimators=200, max_samples=.5).fit(X_train, y_train)
Y_pred = rf.predict(X_test)
print('Accuracy {0}'.format(np.round(accuracy_score(y_test, Y_pred),3)))

Accuracy 0.641


In [12]:
for name, score in zip(X.columns, rf.feature_importances_):
    print(name,np.round(score,3))

DP 0.159
VD 0.064
AF 0.108
HIAF 0.114
REFBIAS 0.143
VARBIAS 0.052
QUAL 0.083
ODDRATIO 0.117
GNB1 0.001
NRAS 0.003
NOTCH2 0.004
LY9 0.0
DNMT3A 0.012
ASXL2 0.003
CERKL 0.001
SF3B1 0.005
TET2 0.013
SNX18 0.002
SYNE1 0.001
AC078842.3 0.0
BRAF 0.004
RAD21 0.005
JAK2 0.002
TET1 0.002
CBL 0.003
KRAS 0.003
PTPN11 0.001
CREBBP 0.005
TP53 0.008
DNAH2 0.002
PPM1D 0.005
ASXL1 0.01
CHMP4B 0.002
GNAS 0.002
LIPI 0.001
U2AF1 0.002
ZFX 0.0
BCOR 0.005
BCORL1 0.004
BRCC3 0.005
SRY 0.001
ZFY 0.0
AMELY 0.0
NOTCH1 0.001
IDH2 0.002
SRSF2 0.0
RP11-661A12.9 0.0
TLK2 0.0
SLC5A10 0.0
Baseline 0.007
Y3 0.007
Male 0.005
Female 0.006
HIGH 0.004
MODERATE 0.004
bias_decimal 0.005


## Bagging Classifier 

In [13]:
clf = BaggingClassifier(base_estimator=SVC(), n_estimators=100, max_samples=.5, random_state=0).fit(X_train, y_train)
Y_pred = clf.predict(X_test)
print('Accuracy {0}'.format(np.round(accuracy_score(y_test, Y_pred),3)))

KeyboardInterrupt: 